In [1]:
DISTRICT_ENTITIES = ["quận 1", "quận 2", "quận 3", "quận 4",
                     "quận 5", "quận 6", "quận 7", "quận 8", "quận 9", "quận 10"]
WARD_ENTITIES = ["phường 1", "phường 2", "phường 3", "phường 4",
                 "phường 5", "phường 6", "phường 7", "phường 8", "phường 9", "phường 10"]
STREET_ENTITIES = ["Lê Lai", "Lê Lợi", "Lê Duẩn"]
HOUSE_ENTITIES = ["cấp 1", "cấp 2", "cấp 3", "cấp 4"]
HOUSE_QUERY_ENTITIES = ["nhà"]


In [2]:
class BaseValue:
    def __init__(self):
        self.type = str
        self._value = None

    @property
    def is_filled(self):
        return self._value is not None

    @classmethod
    def class_name(cls):
        return cls.__name__

    def __repr__(self):
        return f"{self._value}: {self.type.__name__}"

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, _value):
        if _value is not None:
            if isinstance(_value, self.type):
                self._value = _value
            else:
                raise ValueError(
                    f"Value of {self.__class__.__name__} must be instance of {self.type}")


In [3]:
class BaseString(BaseValue):
    def __init__(self):
        super(BaseString, self).__init__()
        self.type = str


In [4]:
class BaseEntity:
    def __repr__(self):
        output = {
            "class": self.__class__.__name__
        }
        for attr in self.attributes:
            output[attr] = self.get_attribute_type(attr)
        return str(output)

    @classmethod
    def class_name(cls):
        return cls.__name__

    @property
    def attributes(self):
        output = self.__dir__()
        return [item for item in output
                if "__" not in item and "_" != item[0]
                and item not in ["attributes", "get_attribute_type", "class_name", "has_attribute", "add_attribute"]]

    def get_attribute_type(self, attr_name):
        if attr_name in self.attributes:
            return self.__getattribute__(attr_name)

    def add_attribute(self, item):
        for attr in self.attributes:
            attr_value = self.get_attribute_type(attr)
            if attr_value.type.__name__ == item.class_name():
                attr_value.value = item
                return True
        return False

    # def add_attribute(self, item):
    #     attr_value = self.get_attribute_type(item.class_name())
    #     attr_value.value = item


In [5]:
class District(BaseEntity):
    def __init__(self, name=None):
        super(District, self).__init__()
        self._name = BaseString()
        self._name.value = name

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, _name):
        self._name.value = _name


In [6]:
class DistrictType(BaseValue):
    def __init__(self):
        super(DistrictType, self).__init__()
        self.type = District


In [7]:
class Ward(BaseEntity):
    def __init__(self, name=None, belongDistrict=None):
        super(Ward, self).__init__()
        self._name = BaseString()
        self._belongDistrict = DistrictType()
        self._name.value = name
        self._belongDistrict.name = belongDistrict

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, _name):
        self._name.value = _name

    @property
    def belongDistrict(self):
        return self._belongDistrict

    @belongDistrict.setter
    def belongDistrict(self, _belongDistrict):
        self._belongDistrict = _belongDistrict


In [8]:
class WardType(BaseValue):
    def __init__(self):
        super(WardType, self).__init__()
        self.type = Ward


In [9]:
class Street(BaseEntity):
    def __init__(self, name=None, belongWard=None):
        super(Street, self).__init__()
        self._name = BaseString()
        self._belongWard = WardType()
        self._name.value = name
        self._belongWard.name = belongWard

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, _name):
        self._name.value = _name

    @property
    def belongWard(self):
        return self._belongWard

    @belongWard.setter
    def belongWard(self, _belongWard):
        self._belongWard = _belongWard


In [10]:
class House:
    def __init__(self, name=None):
        super(House, self).__init__()
        self._name = BaseString()
        self._name.value = name

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, _name):
        self._name.value = _name


In [11]:
class NER:
    @staticmethod
    def extract(input_text):
        label = ""
        output = []

        for ent in DISTRICT_ENTITIES:
            if ent in input_text:
                output.append(District(name=ent))
            else:
                label = "District"

        for ent in WARD_ENTITIES:
            if ent in input_text:
                output.append(Ward(name=ent))
            else:
                label = "Ward"

        for ent in STREET_ENTITIES:
            if ent in input_text:
                output.append(Street(name=ent))
            else:
                label = "Street"

        for ent in HOUSE_ENTITIES:
            if ent in input_text:
                output.append(House(name=ent))
            else:
                label = "House"

        return label, output


In [12]:
text = "Mua nhà ở phường 1 quận 1"
[matchLabel, whereLabel] = NER.extract(text)
print(matchLabel)
print(whereLabel)


House
[{'class': 'District', 'name': quận 1: str}, {'class': 'Ward', 'name': phường 1: str, 'belongDistrict': None: District}]


In [14]:
# class RelationExtraction:
#     @staticmethod
#     def mapping(input_entities: [BaseEntity]):
#         for src_entity in input_entities:
#             for dst_entity in input_entities:
#                 success = src_entity.add_attribute(dst_entity)
#                 print(f"ORIGINAL ENTITY: {src_entity}")
#                 if success:
#                     print(f"MAPPED ENTITY: {src_entity}")
#                 print("++++++++++++++++++++++++++++++++++++++++")


In [15]:
# RelationExtraction.mapping(entities)


In [13]:
import pandas as pd
from neo4j import GraphDatabase

In [14]:
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'password'))

In [15]:
cqlNodeQuery = "match (n:" + matchLabel + "), (n1:Ward), (n2:District) where n1.name = '" + \
    whereLabel[1].name.value + "' and n2.name = '" + \
    whereLabel[0].name.value+"' return n"


In [16]:
def run_query(query):
    print("---query--")
    print(query)
    with driver.session(database="htdb") as session:
        result = session.run(query)
        print("----- result -----")
        # print(type(result))
        for r in result:
            print(r.values())
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [17]:
run_query(cqlNodeQuery)

---query--
match (n:House), (n1:Ward), (n2:District) where n1.name = 'phường 1' and n2.name = 'quận 1' return n
----- result -----
[<Node id=45 labels=frozenset({'House'}) properties={'house_id': '1', 'house_type': 'cấp 1', 'district_id': '1', 'ward_id': '1'}>]
[<Node id=45 labels=frozenset({'House'}) properties={'house_id': '1', 'house_type': 'cấp 1', 'district_id': '1', 'ward_id': '1'}>]
[<Node id=46 labels=frozenset({'House'}) properties={'house_id': '2', 'house_type': 'cấp 2', 'district_id': '1', 'ward_id': '1'}>]
[<Node id=46 labels=frozenset({'House'}) properties={'house_id': '2', 'house_type': 'cấp 2', 'district_id': '1', 'ward_id': '1'}>]
[<Node id=47 labels=frozenset({'House'}) properties={'house_id': '3', 'house_type': 'cấp 3', 'district_id': '1', 'ward_id': '1'}>]
[<Node id=47 labels=frozenset({'House'}) properties={'house_id': '3', 'house_type': 'cấp 3', 'district_id': '1', 'ward_id': '1'}>]
[<Node id=48 labels=frozenset({'House'}) properties={'house_id': '4', 'house_type':

,n


In [ ]:
# cqlNodeQuery = "match (n:House) where n.ward_id = '1' and n.district_id = '1' return n"
# cqlNodeQuery = "match (n:House), (n1:Ward), (n2:District) where n1.name = 'phường 1' and n2.name = 'quận 1' return n"